In [1]:
import os
import json
from typing import TypedDict, Dict

In [2]:
from dotenv import load_dotenv

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

In [4]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END

In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:1.5b",
    temperature=0,
    # other params...
)

In [6]:
class AgentState(TypedDict):
    message: str

# def greeting_node(state: AgentState) -> AgentState:
#     """Function to greet the user and modify the state."""
    
#     state["message"] = "Hello! "+ state["message"] + ", How can I assist you today?"
#     return state

def greeting_node(state: AgentState) -> AgentState:
    """Function to greet any user using LLM."""
    user_message = state['message']
    
    # Extract the actual user message from context if present
    if "User message:" in user_message:
        actual_message = user_message.split("User message:")[-1].strip()
    else:
        actual_message = user_message
    
    # Create a prompt that works for any user introduction
    prompt = f"""
    A user just said: "{actual_message}"
    
    Generate a short, friendly greeting that:
    1. Acknowledges their name if they mentioned it
    2. Asks how you can help them
    3. Is personalized but professional
    
    Examples:
    - If they say "hi I'm John": "Hello John! How can I help you today?"
    - If they say "my name is Sarah": "Nice to meet you Sarah! How may I assist you?"
    
    Only return the greeting, no quotes or explanations.
    """
    
    response = llm.invoke(prompt)
    
    # Clean up the response
    content = response.content
    if '</think>' in content:
        content = content.split('</think>')[-1].strip()
    
    if content.startswith('"') and content.endswith('"'):
        content = content[1:-1]
    
    state["message"] = content.strip()
    return state

In [7]:
graph = StateGraph(AgentState)

graph.add_node("greeter", greeting_node)

graph.set_entry_point("greeter")
graph.set_finish_point("greeter")

app = graph.compile()



In [8]:
context = "You are a helpful assistant. You can answer questions, provide information, and assist with various tasks. Your goal is to be helpful and informative. no emojis, no exclamation marks, no thinking tags, no explanations, just the answer. If you don't know the answer, say 'I don't know'."
message_with_context = f"Context: {context}\nUser message: Hey, this is Dio"


In [ ]:
result = app.invoke({"message": message_with_context})

In [ ]:
result["message"]

'Hello [Name]! How can I assist you today?'